## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rio De Janeiro,BR,-22.90,-43.21,75.20,78,100,13.87,moderate rain
1,1,Santa Isabel Do Rio Negro,BR,-0.41,-65.02,78.06,83,83,1.43,light rain
2,2,Honiara,SB,-9.43,159.95,87.80,70,20,6.93,few clouds
3,3,Misratah,LY,32.38,15.09,82.89,57,0,8.50,clear sky
4,4,Torbay,CA,47.67,-52.73,66.00,68,90,13.87,overcast clouds


In [49]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [50]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
nice_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
nice_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rio De Janeiro,BR,-22.90,-43.21,75.20,78,100,13.87,moderate rain
1,1,Santa Isabel Do Rio Negro,BR,-0.41,-65.02,78.06,83,83,1.43,light rain
2,2,Honiara,SB,-9.43,159.95,87.80,70,20,6.93,few clouds
3,3,Misratah,LY,32.38,15.09,82.89,57,0,8.50,clear sky
8,8,Atuona,PF,-9.80,-139.03,78.66,76,8,16.98,light rain


In [51]:
# 4a. Determine if there are any empty rows.
nice_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261 entries, 0 to 717
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              261 non-null    int64  
 1   City                 261 non-null    object 
 2   Country              261 non-null    object 
 3   Lat                  261 non-null    float64
 4   Lng                  261 non-null    float64
 5   Max Temp             261 non-null    float64
 6   Humidity             261 non-null    int64  
 7   Cloudiness           261 non-null    int64  
 8   Wind Speed           261 non-null    float64
 9   Current Description  261 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 22.4+ KB


In [52]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
nice_cities_df = nice_cities_df.dropna()

In [53]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = nice_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rio De Janeiro,BR,75.20,moderate rain,-22.90,-43.21,
1,Santa Isabel Do Rio Negro,BR,78.06,light rain,-0.41,-65.02,
2,Honiara,SB,87.80,few clouds,-9.43,159.95,
3,Misratah,LY,82.89,clear sky,32.38,15.09,
8,Atuona,PF,78.66,light rain,-9.80,-139.03,
13,Cockburn Town,TC,84.20,scattered clouds,21.46,-71.14,
15,Tomatlan,MX,77.88,moderate rain,19.93,-105.25,
26,Kavaratti,IN,81.28,light rain,10.57,72.64,
28,Dicabisagan,PH,81.34,overcast clouds,17.08,122.42,
31,Sakakah,SA,78.80,clear sky,29.97,40.21,


In [54]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [55]:
# Check for any null hotel values before step 7.
hotel_df["Hotel Name"].isnull().sum()

0

In [56]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0, how="any")
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rio De Janeiro,BR,75.20,moderate rain,-22.90,-43.21,Windsor Guanabara Hotel
1,Santa Isabel Do Rio Negro,BR,78.06,light rain,-0.41,-65.02,Hotel GSA
2,Honiara,SB,87.80,few clouds,-9.43,159.95,Heritage Park Hotel
3,Misratah,LY,82.89,clear sky,32.38,15.09,فندق الماسة
8,Atuona,PF,78.66,light rain,-9.80,-139.03,Villa Enata
...,...,...,...,...,...,...,...
712,Rudbar,AF,75.52,clear sky,30.15,62.60,
713,Sterling,US,78.01,clear sky,39.01,-77.43,Hampton Inn & Suites Washington-Dulles Interna...
714,Bawku,GH,77.99,broken clouds,11.06,-0.24,Ultimate Guest House
716,Cameta,BR,83.21,broken clouds,-2.24,-49.50,Hotel San Juan


In [57]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [58]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))